In [1]:
import pandas as pd
import re
import os
import sys
import pickle as pkl
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import json

# df_annotations = pd.read_csv("all_annotations.csv")


# Get data

In [ ]:
# with open("protocols.pkl.gz", "rb") as f:
#     file_to_pages = pkl.load(f)

In [ ]:
# import spacy
# nlp = spacy.blank("en")

In [ ]:
# texts = []

# for i in range(len(df_annotations)):
#     file_name = df_annotations.file.iloc[i]
    
#     pages= file_to_pages[file_name]
    
#     text = ""
#     for page in pages:
#         doc = nlp(page)
        
#         is_include = [False] * len(doc)
#         for tok in doc:
#             if tok.text.lower() in {"phase"}:
#                 for token_index in range(tok.i - 20, tok.i + 20):
#                     if token_index >= 0 and token_index < len(doc):
#                         is_include[token_index] = True
        
#         for token_index in range(len(doc)):
#             if is_include[token_index]:
#                 text += doc[token_index].text + doc[token_index].whitespace_
                
                
#     if text == "":
#         print ("nothing found", file_name, df_annotations.phase.iloc[i])
#         text = " ".join(pages)
#         if len(text) > 100000:
#             text = text[:100000]
#     else:
#         print ("found", file_name, df_annotations.phase.iloc[i])

#     texts.append(text)
# df_annotations["text"] = texts

In [ ]:
# df_annotations.to_csv("filtered_for_phase.csv.bz2")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [100]:
df_annotations= pd.read_csv("/content/drive/MyDrive/data/filtered_for_phase_arms_subjects.csv.bz2")

In [101]:
(~df_annotations.num_subjects.isna()).sum(), (~annots_with_subject.num_subjects.isna()).sum()

(418, 11920)

Override the subjects column with more up to date data

In [104]:
annots_with_subject = pd.read_csv("/content/drive/MyDrive/data/all_annotations.csv")

In [105]:
df_annotations["num_subjects"] = annots_with_subject["num_subjects"]

In [106]:
def get_num_subjects_clean(num):
    if pd.isna(num):
        return None
    if num >= 10000:
        return "10000+"
    if num >= 1000:
        return "1000+"
    if num >= 500:
        return "500+"
    if num >= 200:
        return "200+"
    if num >= 100:
        return "100+"
    if num >= 50:
        return "50+"
    if num >= 25:
        return "25+"
    return "1-24"
df_annotations["num_subjects_clean"] = df_annotations["num_subjects"].apply(get_num_subjects_clean)

In [107]:
df_annotations["num_subjects_clean"].value_counts()

1-24      2980
25+       2119
50+       2016
100+      1674
200+      1656
500+       816
1000+      547
10000+     112
Name: num_subjects_clean, dtype: int64

In [108]:
# df_annotations = pd.read_csv("filtered_for_phase.csv.bz2")

In [109]:
# del file_to_pages

In [110]:
phase_map = {"Phase 2":"2",
"Phase 3":"3",
"Phase 4":"4",
"Phase 1":"1",
"Phase 1/Phase 2":"1.5",
"Not Applicable":"0",
"Phase 2/Phase 3":"2.5",
"Early Phase 1":"0.5"}
df_annotations["phase_clean"] = df_annotations["phase"].apply(lambda x : phase_map.get(x, None))

In [111]:
def get_num_arms_clean(num):
    if pd.isna(num):
        return None
    if num >= 5:
        num = 5
    return num
df_annotations["num_arms_clean"] = df_annotations["num_arms"].apply(get_num_arms_clean)

In [112]:
df_train = df_annotations[df_annotations.train_val == "train"]
df_val = df_annotations[df_annotations.train_val == "val"]

In [113]:
# TRAINING_DATA = [
#     ["My little kitty is so special", {"KAT0": True}],
#     ["Dude, Totally, Yeah, Video Games", {"KAT1": True}],
#     ["Should I pay $1,000 for the iPhone X?", {"KAT1": True}],
#     ["The iPhone 8 reviews are here", {"KAT1": True}],
#     ["Noa is a great cat name.", {"KAT0": True}],
#     ["We got a new kitten!", {"KAT0": True}]
# ]

TRAINING_DATA = []
for idx in range(len(df_train)):
    cats = {}
    for a in phase_map.values():
        cats[f"phase{a}"] = 0
    for a in range(1, 6):
        cats[f"arms{a}"] = 0
    for a in set(df_annotations["num_subjects_clean"]):
        cats[f"subjects{a}"] = 0
    if not pd.isna(df_train.phase_clean.iloc[idx]):
      cats[f"phase{df_train.phase_clean.iloc[idx]}"] = 1
    if not pd.isna(df_train.num_arms_clean.iloc[idx]):
      cats[f"arms{int(df_train.num_arms_clean.iloc[idx])}"] = 1
    if not pd.isna(df_train.num_subjects_clean.iloc[idx]):
      cats[f"subjects{df_train.num_subjects_clean.iloc[idx]}"] = 1

    text = df_train.text.iloc[idx]
    if len(text) > 1000000:
        text = text[:1000000]
    
    TRAINING_DATA.append([text , cats])

In [116]:
import spacy
# Add imports for example, as well as textcat config...
from spacy.training import Example
from spacy.pipeline.textcat_multilabel import multi_label_default_config
from thinc.api import Config
import random

# labels should be one-hot encoded



# bow
# config = Config().from_str(single_label_bow_config)

# textensemble with attention
config = Config().from_str(multi_label_default_config)

nlp = spacy.blank("en")
# now uses `add_pipe` instead
category = nlp.add_pipe("textcat_multilabel", last=True, config=config)
for a in range(1, 6):
    category.add_label(f"arms{a}")
for a in phase_map.values():
    category.add_label(f"phase{a}")
for a in set(df_annotations["num_subjects_clean"]):
    category.add_label(f"subjects{a}")
# Start the training
nlp.begin_training()

# Loop for 10 iterations
for itn in range(100):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA)
    losses = {}

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA, size=4):
        texts = [nlp.make_doc(text) for text, entities in batch]
        annotations = [{"cats": entities} for text, entities in batch]

        # uses an example object rather than text/annotation tuple
        examples = [Example.from_dict(doc, annotation) for doc, annotation in zip(
            texts, annotations
        )]
        nlp.update(examples, losses=losses)
    if itn % 20 == 0:
        print(losses)


{'textcat_multilabel': 202.14193032681942}


KeyboardInterrupt: ignored

In [ ]:
predictions = []
for idx in range(len(df_val)):
    doc = nlp(df_val.text.apply(str).iloc[idx])
    predictions.append(doc.cats)

In [ ]:
def get_arms_prediction(p):
  arms_dict = dict([int(re.sub(r'arms', '', k)), v] for k, v in p.items() if "arms" in k)
  return max(arms_dict, key=arms_dict.get)

In [ ]:
df_val["y_pred_arms"] = [get_arms_prediction(p) for p in predictions]

In [ ]:
def get_subjects_prediction(p):
  subjects_dict = dict([re.sub(r'subjects', '', k), v] for k, v in p.items() if "subjects" in k)
  return max(subjects_dict, key=subjects_dict.get)

In [ ]:
df_val["y_pred_num_subjects"] = [get_subjects_prediction(p) for p in predictions]

In [ ]:
def get_phase_prediction(p):
  phase_dict = dict([float(re.sub(r'phase', '', k)), v] for k, v in p.items() if "phase" in k)
  return max(phase_dict, key=phase_dict.get)

In [ ]:
df_val["y_pred_phase"] = [get_phase_prediction(p) for p in predictions]

In [ ]:
acc = accuracy_score(df_val.phase_clean, df_val["y_pred_phase"])
print (f"Phase accuracy {acc}")

In [ ]:
ConfusionMatrixDisplay.from_predictions(df_val.phase_clean, df_val["y_pred_phase"])
plt.xticks(rotation=90)
;

In [ ]:
acc = accuracy_score(df_val.num_subjects_clean, df_val["y_pred_num_subjects"])
print (f"Subjects accuracy {acc}")

In [ ]:
ConfusionMatrixDisplay.from_predictions(df_val.num_subjects_clean, df_val["y_pred_num_subjects"])
plt.xticks(rotation=90)
;

In [ ]:
acc = accuracy_score(df_val.num_arms_clean, df_val["y_pred_arms"])
print (f"Arms accuracy {acc}")

In [ ]:
ConfusionMatrixDisplay.from_predictions(df_val.num_arms_clean, df_val["y_pred_arms"])
plt.xticks(rotation=90)
;

In [ ]:
nlp.to_disk("phase_arms_subjects_model_04_textcat.spacy")


In [ ]:
nlp.to_disk("/content/drive/MyDrive/data/phase_arms_subjects_model_04_textcat.spacy")
